<a href="https://colab.research.google.com/github/proygit/DMT/blob/main/EDA_Expedia_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import all libraries

In [ ]:
!pip install pandasql


In [3]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
import sqlite3
from pandasql import sqldf

#Mount Drive

In [16]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Change the working directory to the desired folder
os.chdir('/content/drive/MyDrive/DMT/data')

# List the contents of the current working directory
contents = os.listdir()

# Print the contents
print(contents)
data_path = "/content/drive/My Drive/DMT/data/"

['submission_sample.csv', 'test.csv', 'train.csv', 'train.db', 'df_train.db']


#Train data set-EDA

In [31]:

df_train = pd.read_csv(data_path + "train.csv")
#df_train=df_train.dropna(axis=1,how="any")
#df_train = pd.read_csv(data_path + "train.csv",nrows=50000)
df_train.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


#Original Data set had 54 rows,after dropping all columns which had Nan values we are left with 23

In [129]:
df_train.shape

(4958347, 54)

In [96]:
#check missing values in 23 column
df_train.isnull().sum()

srch_id                           0
date_time                         0
site_id                           0
visitor_location_country_id       0
visitor_hist_starrating        9503
visitor_hist_adr_usd           9503
prop_country_id                   0
prop_id                           0
prop_starrating                   0
prop_review_score                17
prop_brand_bool                   0
prop_location_score1              0
prop_location_score2           2269
prop_log_historical_price         0
position                          0
price_usd                         0
promotion_flag                    0
srch_destination_id               0
srch_length_of_stay               0
srch_booking_window               0
srch_adults_count                 0
srch_children_count               0
srch_room_count                   0
srch_saturday_night_bool          0
srch_query_affinity_score      9460
orig_destination_distance      3204
random_bool                       0
comp1_rate                  

In [97]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 54 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   srch_id                      10000 non-null  int64  
 1   date_time                    10000 non-null  object 
 2   site_id                      10000 non-null  int64  
 3   visitor_location_country_id  10000 non-null  int64  
 4   visitor_hist_starrating      497 non-null    float64
 5   visitor_hist_adr_usd         497 non-null    float64
 6   prop_country_id              10000 non-null  int64  
 7   prop_id                      10000 non-null  int64  
 8   prop_starrating              10000 non-null  int64  
 9   prop_review_score            9983 non-null   float64
 10  prop_brand_bool              10000 non-null  int64  
 11  prop_location_score1         10000 non-null  float64
 12  prop_location_score2         7731 non-null   float64
 13  prop_log_historic

In [98]:
df_train.columns

Index(['srch_id', 'date_time', 'site_id', 'visitor_location_country_id',
       'visitor_hist_starrating', 'visitor_hist_adr_usd', 'prop_country_id',
       'prop_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool',
       'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
       'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
       'srch_adults_count', 'srch_children_count', 'srch_room_count',
       'srch_saturday_night_bool', 'srch_query_affinity_score',
       'orig_destination_distance', 'random_bool', 'comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate'

#Drop columns  date_time,site_id

In [99]:
#train=train.drop(["srch_id","date_time","site_id","visitor_location_country_id","prop_country_id","prop_id","srch_destination_id"],axis=1)
df_train=df_train.drop(["date_time","site_id"],axis=1)

In [100]:
df_train.shape
#df_train.columns

(10000, 52)

In [101]:
search_id_counts = df_train['srch_id'].value_counts()
#search_id_counts

In [102]:
prop_id_counts = df_train['prop_id'].value_counts()
#prop_id_counts

In [103]:
df_train.columns

Index(['srch_id', 'visitor_location_country_id', 'visitor_hist_starrating',
       'visitor_hist_adr_usd', 'prop_country_id', 'prop_id', 'prop_starrating',
       'prop_review_score', 'prop_brand_bool', 'prop_location_score1',
       'prop_location_score2', 'prop_log_historical_price', 'position',
       'price_usd', 'promotion_flag', 'srch_destination_id',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool',
       'srch_query_affinity_score', 'orig_destination_distance', 'random_bool',
       'comp1_rate', 'comp1_inv', 'comp1_rate_percent_diff', 'comp2_rate',
       'comp2_inv', 'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_ra

In [131]:
df_train.shape

(4958347, 54)

In [ ]:
df_train['click_bool']

#Apply the query for NDGC@5

1.  First create an sqllite connection and test a query for a smaller data set








In [32]:
conn = sqlite3.connect('df_train.db')

In [33]:
# Register the DataFrame as a temporary table for small data set
df_train.to_sql('tmp_table', conn, if_exists='replace', index=False)

4958347

In [34]:
query1 = 'SELECT prop_id FROM tmp_table WHERE prop_review_score > 3.5'

In [29]:
query2 = '''
SELECT 
    srch_id,
    prop_id,
    CASE
        WHEN click_bool AND booking_bool THEN 5
        WHEN click_bool THEN 1
        ELSE 0
    END AS relevance
FROM tmp_table
'''

In [45]:
query3 = '''
SELECT 
    srch_id,
    prop_id,prop_starrating,prop_review_score,visitor_location_country_id,
    CASE
        WHEN click_bool AND booking_bool THEN 5
        WHEN click_bool THEN 1
        ELSE 0
    END AS relevance
FROM tmp_table
'''

In [ ]:

result_df = pd.read_sql_query(query3, conn)
# Print the result
print(result_df)

#Check which features contribute to Relevance of 5 by looking at the mean

In [ ]:
# Execute query3 and load results into a DataFrame
result_df = pd.read_sql_query(query3, conn)

# Group by relevance and calculate mean for each feature
mean_by_relevance = result_df.groupby('relevance').mean()

# Print mean values for features with relevance = 5
print(mean_by_relevance.loc[5])


In [47]:
# Count the unique values in the relevance column
print(result_df['relevance'].value_counts())


0    4736468
5     138390
1      83489
Name: relevance, dtype: int64


In [38]:
# Calculate DCG@5 and IDCG@5 for each search query
ndcg_list = []
for search_id, group in result_df.groupby('srch_id'):
    # Sort the properties by relevance score in descending order
    sorted_group = group.sort_values('relevance', ascending=False)[:5]
    
    # Calculate DCG@5
    dcg = np.sum(sorted_group['relevance'] / np.log2(np.arange(2, 7)))
    
    # Calculate IDCG@5
    idcg = np.sum(sorted(group['relevance'], reverse=True)[:5] / np.log2(np.arange(2, 7)))
  
    # Calculate NDCG@5
    ndcg = dcg / idcg if idcg > 0 else 0
    
    ndcg_list.append(ndcg)

# Calculate the average NDCG@5 over all search queries
avg_ndcg = np.mean(ndcg_list)
print(avg_ndcg)


1.0


In [39]:
# Count the unique values in the relevance column
print(result_df['relevance'].value_counts())


0    4736468
5     138390
1      83489
Name: relevance, dtype: int64


In [40]:
correlation_matrix = result_df.corr()
print(correlation_matrix)


            srch_id   prop_id  relevance
srch_id    1.000000 -0.000166   0.000434
prop_id   -0.000166  1.000000  -0.000342
relevance  0.000434 -0.000342   1.000000


#Looking for booking_bool == True

In [ ]:

booked = df_train[df_train['booking_bool'] == True].value_counts()
booked.head()

#Look up for the most common property_id when booking was True,belwo code gives the first 5 index

In [ ]:
#booked_prop_ids = list(booked.prop_id.value_counts().head().index)
#booked_prop_ids


In [ ]:
#Get DataFrame where booking is True
df_train_setBooking = df_train[df_train.booking_bool == True]
#df_train_setBooking

In [ ]:

df_train_setBooking.shape

#Apply Co-relation and generate Heat map to the co-relation between the prop_id and other variables

In [ ]:
df_train_setBooking.corr()["prop_id"]

In [ ]:

mask = df_train_setBooking['booking_bool'] == True
corr_matrix = df_train_setBooking[mask].drop(columns=['click_bool']).corr()

sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, vmin=-1, vmax=1)


In [ ]:
columns = ['srch_id', 'visitor_location_country_id', 'prop_country_id', 'prop_id',
           'prop_starrating', 'prop_brand_bool', 'prop_location_score1',
           'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
           'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window',
           'srch_adults_count', 'srch_children_count', 'srch_room_count',
           'srch_saturday_night_bool', 'random_bool', 'click_bool',
           'booking_bool']

# Select only the specified columns from the dataframe
df_selected_columns = df_train_setBooking[columns]

# Compute the correlation matrix
corr = df_selected_columns.corr()

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(20, 10))

# Draw the heatmap using seaborn
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, ax=ax)

# Add a title to the plot
ax.set_title('Correlation Heatmap')

# Show the plot
plt.show()
# Sort the correlations with prop_id in descending order
corr_prop_id = corr['prop_id'].sort_values(ascending=False)
print(corr_prop_id)

In [ ]:
columns = ['srch_id', 'visitor_location_country_id',  'prop_id',
           'prop_starrating',
           'prop_log_historical_price', 'position', 'price_usd', 'promotion_flag',
           'srch_destination_id', 'srch_length_of_stay',
           'srch_room_count',
           'srch_saturday_night_bool', 
           ]

# Select only the specified columns from the dataframe
df_selected_columns = df_train_setBooking[columns]

# Compute the correlation matrix
corr = df_selected_columns.corr()

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(20, 10))

# Draw the heatmap using seaborn
sns.heatmap(corr, annot=True, cmap='coolwarm', linewidths=0.5, ax=ax)

# Add a title to the plot
ax.set_title('Correlation Heatmap')

# Show the plot
plt.show()

#Apply PCA

#Forwards Sequential Feature Selection on Training data

In [ ]:

# Define the input features and the target variable
input_features = ['srch_id', 'visitor_location_country_id', 
                  'prop_starrating', 'prop_log_historical_price', 
                  'position', 'price_usd', 'promotion_flag',
                  'srch_destination_id', 'srch_length_of_stay',
                  'srch_room_count', 'srch_saturday_night_bool']

target_variable = 'prop_id'

# Set the number of folds for cross-validation
n_folds = 3

# Load the data and select the first 1000 rows
X = df_train_setBooking[input_features].iloc[:1000]
y = df_train_setBooking[target_variable].iloc[:1000]


# Initialize the k-fold object
kf = KFold(n_splits=n_folds, shuffle=True)

# Initialize the model
rf = RandomForestClassifier()

# Initialize a list to store the cross-validation scores
cv_scores = []

# Loop over the folds
for train_idx, val_idx in kf.split(X):
    # Split the data into training and validation sets for the current fold
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    # Fit the model on the training data
    rf.fit(X_train, y_train)
    
    # Evaluate the model on the validation data and append the score to the list of cross-validation scores
    y_pred = rf.predict(X_val)
    cv_scores.append(accuracy_score(y_val, y_pred))

# Compute the average cross-validation score
avg_cv_score = sum(cv_scores) / len(cv_scores)

# Print the average cross-validation score
print('Average cross-validation score: {:.4f}'.format(avg_cv_score))




    


In [ ]:
# Define the input features and the target variable
input_features = ['srch_id', 'visitor_location_country_id', 
                  'prop_starrating', 'prop_log_historical_price', 
                  'position', 'price_usd', 'promotion_flag',
                  'srch_destination_id', 'srch_length_of_stay',
                  'srch_room_count', 'srch_saturday_night_bool']

target_variable = 'prop_id'

# Set the number of folds for cross-validation
n_folds = 5

# Load the data and select the first 1000 rows
X = df_train_setBooking[input_features].iloc[:1000]
y = df_train_setBooking[target_variable].iloc[:1000]

# Initialize the k-fold object
kf = KFold(n_splits=n_folds, shuffle=True)

# Initialize the model
rf = RandomForestClassifier()

# Initialize a list to store the cross-validation scores
cv_scores = []

# Loop over the folds
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
    # Split the data into training and validation sets for the current fold
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    # Fit the model on the training data
    rf.fit(X_train, y_train)
    
    # Evaluate the model on the validation data and append the score to the list of cross-validation scores
    y_pred = rf.predict(X_val)
    cv_score = accuracy_score(y_val, y_pred)
    cv_scores.append(cv_score)
    
    # Print the accuracy for the current fold
    print('Fold {}: {:.4f}'.format(fold_idx + 1, cv_score))

# Compute the average cross-validation score
avg_cv_score = sum(cv_scores) / len(cv_scores)

# Print the average cross-validation score
print('Average cross-validation score: {:.4f}'.format(avg_cv_score))


#Test Data Set EDA

In [ ]:
df_test = pd.read_csv(data_path + "test.csv")
#Drop all columns with Nan values
df_test=df_test.dropna(axis=1,how="any")
#original had 54 now we have only 20
df_test.shape


In [ ]:
df_test.isnull().sum()

In [ ]:
df_test.info()

In [ ]:
df_test.columns